In [5]:
import numpy as np
import os
import glob
import random
import json
import matplotlib.pyplot as plt
from obspy import read
from datetime import datetime, timedelta
from tqdm import tqdm

In [106]:
# 取得某個階段的所有檔名
def find_wave(year, month, day, hour, minute, sec, wave_path, waveFile):
    month = str(month) if month//10==1 else '0'+str(month)
    day = '0'+str(day) if day//10==0 else str(day)
    
    target = wave_path + '/' + str(year)+month+day + '*/' + waveFile    

    result = glob.glob(target)
    print(result)
    if len(result) > 1:
        hour = '0'+str(hour) if hour//10==0 else str(hour)
        minute = '0'+str(minute) if minute//10==0 else str(minute)
        sec = '0'+str(sec) if sec//10==0 else str(sec)
        
        result = glob.glob(wave_path + '/' + str(year)+month+day + '*/' + waveFile  )
    return 

In [7]:
def getFolder(year, month, day, hour, minute, sec):
    month = str(month) if month/10>=1 else '0'+str(month)
    day = str(day) if day/10>=1 else '0'+str(day)
    hour = str(hour) if hour/10>=1 else '0'+str(hour)
    minute = str(minute) if minute/10>=1 else '0'+str(minute)
    sec = str(sec) if sec/10>=1 else '0'+str(sec)
    
    return str(year)+month+day+'_'+hour+minute+sec

In [145]:
def getWavFile(infile, root_path):
    
    with open(infile) as f:
        lines = f.readlines()
        
    adict = {}
    adict['network'] = infile[-7:-4]
    adict['RecordLength(sec)'] = float(lines[3][-9:])
    adict['sampling_rate'] = int(lines[4][-5:])
    adict['datatype'] = 'Acceleration'
    
    return adict

    # load waveform
#     z, n, e = [], [], []
#     for i in lines[11:]:
#         z.append(float(i[14:20]))
#         n.append(float(i[24:30]))
#         e.append(float(i[34:40]))
        
#     z, n, e = np.array(z), np.array(n), np.array(e)
    
    # write to txt
#     filepath = os.path.join(root_path, 'test')
#     print(filepath)
#     with open(filepath+'.txt', 'w') as file:
#         file.write('\tZ\tN\tE\n')
#         for i in range(z.shape[0]):
#             file.write('\t{}\t{}\t{}\n'.format(z[i], n[i], e[i]))

#     return z, n, e, adict['sampling_rate']

In [9]:
def genDuration(start, ptime, stime, sample_rate, length):
    # time: str -> datetime.datetime
    start, ptime, stime = toTime(start), toTime(ptime), toTime(stime)
    
    # ptime, stime 減去儀器開始時間，以得到具體在哪個 sample point
    diff_p, diff_s = ptime-start, stime-start
    sec_diff_p, microsec_diff_p = diff_p.seconds, diff_p.microseconds
    sec_diff_s, microsec_diff_s = diff_s.seconds, diff_s.microseconds
    
    p_trigger_sample = int((sec_diff_p+microsec_diff_p/1e+5)*sample_rate)
    s_trigger_sample = int((sec_diff_s+microsec_diff_s/1e+5)*sample_rate)
   
    # 生成 ground-truth
    gt = np.zeros(length)
    gt[p_trigger_sample:s_trigger_sample] = 1
    
    return gt

In [10]:
def toTime(time):
    year, month, day = int(time[:4]), int(time[5:7]), int(time[8:10])
    hour, minute, second = int(time[11:13]), int(time[14:16]), int(time[17:19])

    if len(time) > 19:
        microsec = int(time[20:-1])
        return datetime(year, month, day, hour, minute, second, microsec)
    else:
        return datetime(year, month, day, hour, minute, second)
    
def fileTime(time):
    year, month, day = time[:4], time[5:7], time[8:10]
    hour, minute, second = time[11:13], time[14:16], time[17:19]
    
    return year+month+day+hour+minute+second

In [11]:
def toCSV(z, n, e, gt, ori_time, station, root_save_path):
    to_write = np.vstack((e, n, z, gt))
    ori_time = fileTime(ori_time)
   
    # filename: 地震發生時間 + 測站名 + 儀器種類
    file_name = ori_time + '_' + station
    save_path = os.path.join(root_save_path, file_name)
#     print(save_path)
    np.savetxt(save_path+'.csv', to_write, delimiter=',')

In [175]:
def unpackP(infile, root_path, wave_path):
    with open(infile) as f:
        lines = f.readlines()
    
    # ================================ header ================================= #
    tmp = lines[0]
    year = int(tmp[1:5])
    month = int(tmp[5:7])
    day = int(tmp[7:9])
    hour = int(tmp[9:11])
    minute = int(tmp[11:13])
    sec = float(tmp[13:19])
    dt = datetime(year,month,day,hour,minute,int(sec//1),int(sec%1 * 1000000))
    
    lat = int(tmp[19:21])
    lat_m = float(tmp[21:26])
    lon = int(tmp[26:29])
    lon_m = float(tmp[29:34])
   
    lat+=lat_m/60
    lat=round(lat,2)
    lon+=lon_m/60
    lon=round(lon,2)
    
    depth = float(tmp[34:40])
    mag = float(tmp[40:44])
    n_sta = int(tmp[-4:])+1
    
    pfile_info = {}
    pfile_info["ori_time"] = str(dt)
    pfile_info["depth"] = depth
    pfile_info["mag"] = mag
    pfile_info["lat"] = lat
    pfile_info["lon"] = lon
    # ================================ header ================================= #
    # ================================= body ================================== #
    for i in range(1, n_sta):
        tmp = lines[i]
        sta = tmp[:7].strip()
        
        try:
            pdict = {}
            adict = {}
            
            starttime = datetime(year=int(tmp[84:88]), month=int(tmp[88:90]), day=int(tmp[90:92]), hour=int(tmp[92:94]),
                                         minute=int(tmp[94:96]), second=int(tmp[96:98]), microsecond=int(tmp[99:101]))
            pdict['starttime'] = str(starttime)
            pdict['distance'] = float(tmp[8:14])
            pdict['Azimuth'] = int(tmp[14:17])
            pdict['p_polar'] = str(tmp[21])
            pdict['p_arrival_time'] = str(starttime + timedelta(seconds=float(tmp[23:30])))
            pdict['p_weight'] = int(tmp[31])
            pdict['s_arrival_time'] = str(starttime + timedelta(seconds=float(tmp[33:40])))
            pdict['s_weight'] = int(tmp[41])
            pdict['station_lat'] = float(tmp[42:50])
            pdict['station_lon'] = float(tmp[50:57])
            
            if pdict['p_polar'] == ' ':
                pdict['p_polar'] = 'unknown' 
                
            # instrument status
            if tmp[80] == '1':
                pdict['Z'] = False
            else:
                pdict['Z'] = True
                
            if tmp[81] == '1':
                pdict['N'] = False
            else:
                pdict['N'] = True
            
            if tmp[82] == '1':
                pdict['E'] = False
            else:
                pdict['E'] = True
            
            # 0: normal, 1: 可能有錯誤，但波型還是可以用, 2: 沒辦法判斷時間系統有沒有問題
            if tmp[83] == '!':
                pdict['time_system'] = 0
            elif tmp[83] == '*':
                pdict['time_system'] = 1
            elif tmp[83] == ' ':
                pdict['time_system'] = 2
                
            # 只挑完美的 waveform
#             if not pdict['Z'] or not pdict['N'] or not pdict['E'] and pdict['time_system'] != 0:
#                 continue
                
            # Store wave file
            waveFile = sta + '_' + tmp[63:75].strip() + '.txt'
            wave = find_wave(year, month, day, hour, minute, sec, wave_path, waveFile)
            
            # 找不到波型檔案
            if not wave:
                pdict['numberOfData'] = 0
            else:
                adict = getWavFile(wave, root_path)
                pdict['waveFile'] = wave
                pdict['0'] = adict   
                pdict['numberOfData'] = 1
                
                
            # write into csv file
#             gt = genDuration(pdict['starttime'], pdict['p_arrival_time'], pdict['s_arrival_time'], sample_rate, z.shape[0])

            #toCSV(z, n, e, gt, pfile_info["ori_time"], sta, '/mnt/nas7/weiwei/earthquake_RNN/newTrain/seis')
#             draw(z, n, e, gt)
            
        except Exception as e:
            print(e)
#             pass

        pfile_info[sta] = pdict
    
    return pfile_info

In [57]:
def draw(z, n, e, gt):
    plt.figure(figsize=(35, 25))
    plt.subplot(411)
    plt.title('Z')
    plt.plot(z)
    
    plt.subplot(412)
    plt.title('N')
    plt.plot(n)
    
    plt.subplot(413)
    plt.title('E')
    plt.plot(e)
    
    plt.subplot(414)
    plt.title('GT')
    plt.plot(gt)
    plt.show()

In [189]:
all_year = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in all_year:
    root_path = "/mnt/nas6/ori_TSMIP/event_info/" + year
    wave_path = "/mnt/nas6/ori_TSMIP/event_info/wave"
    save_path = '/mnt/nas6/TSMIP/'+year
    files = os.listdir(root_path)

    if not os.path.exists(save_path):
        print('creating path: ', save_path)
        os.mkdir(save_path)

    for file in tqdm(files):
        version = file[-1]
        if version == 'P':
            version = '0'

        p = unpackP(os.path.join(root_path, file), root_path, wave_path)

        with open(os.path.join(save_path, file[:8])+'('+version+').json', 'w') as f:
            json.dump(p, f)

creating path:  /mnt/nas6/TSMIP/2012


 75%|████████████████████████████████████████████████████████████████████████████████▌                           | 905/1214 [02:42<00:28, 10.72it/s]

month must be in 1..12


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1214/1214 [04:00<00:00,  5.05it/s]


creating path:  /mnt/nas6/TSMIP/2013


...

creating path:  /mnt/nas6/TSMIP/2018


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 3560/3560 [14:09<00:00,  4.19it/s]


creating path:  /mnt/nas6/TSMIP/2019


  4%|████▍                                                                                                        | 69/1696 [00:28<11:22,  2.38it/s]


ValueError: second must be in 0..59

In [131]:
# 取得某個階段的所有檔名
def find_wave(year, month, day, hour, minute, sec, wave_path, waveFile):
    month = str(month) if month//10==1 else '0'+str(month)
    day = '0'+str(day) if day//10==0 else str(day)
    
    target = wave_path + '/' + str(year)+month+day + '*/' + waveFile    

    result = glob.glob(target)
    
    if len(result) > 1 or len(result) == 0:
        hour = '0'+str(hour) if hour//10==0 else str(hour)
        minute = '0'+str(minute) if minute//10==0 else str(minute)
        sec = '0'+str(sec) if sec//10==0 else str(sec)
        sec = sec[:2]
        
        result = glob.glob(wave_path + '/' + str(year)+month+day+'_'+hour+minute+sec + '/' + waveFile)
#         print('target: ',wave_path + '/' + str(year)+month+day+'_'+hour+minute+sec + '/' + waveFile)
#         print('sec result: ', result)

    if len(result) == 0:
        return 
    
    return result[0]